# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil editar e modificar um pdf
- Mesmo assim o Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [1]:
import PyPDF2 as pyf
from pathlib import Path

arquivo_pdf = pyf.PdfReader(r'Documentos PDF - Mentoria\MGLU_ER_3T20_POR.pdf')

i = 1
for pagina in arquivo_pdf.pages:
    arquivo_novo = pyf.PdfWriter()
    arquivo_novo.add_page(pagina)
    with Path(fr'Documentos PDF - Mentoria\Páginas Separadas\Arquivo{i}.pdf').open(mode='wb') as arquivo_final:
        arquivo_novo.write(arquivo_final)
        i += 1

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [2]:
paginas = [1, 14, 16]

novo_pdf = pyf.PdfWriter()
for num_pagina in paginas:
    arquivo_pdf = pyf.PdfReader(fr'Documentos PDF - Mentoria\Páginas Separadas\Arquivo{num_pagina}.pdf')
    pagina = arquivo_pdf.pages[0]
    novo_pdf.add_page(pagina)

with Path(r'Documentos PDF - Mentoria\Consolidado.pdf').open(mode='wb') as arquivo_final:
        novo_pdf.write(arquivo_final)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [3]:
pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append(r'Documentos PDF - Mentoria\MGLU_ER_3T20_POR.pdf')
pdf_mesclado.append(r'Documentos PDF - Mentoria\MGLU_ER_4T20_POR.pdf')

with Path(r'Documentos PDF - Mentoria\ConsolidadoAppend.pdf').open(mode='wb') as arquivo_final:
        pdf_mesclado.write(arquivo_final)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [4]:
pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append(r'Documentos PDF - Mentoria\MGLU_ER_3T20_POR.pdf')
pdf_mesclado.merge(1, r'Documentos PDF - Mentoria\MGLU_ER_4T20_POR.pdf')

with Path(r'Documentos PDF - Mentoria\ConsolidadoMerge.pdf').open(mode='wb') as arquivo_final:
        pdf_mesclado.write(arquivo_final)

- Rodar Página

In [5]:
pdf_rodar = pyf.PdfReader(r'Documentos PDF - Mentoria\MGLU_ER_3T20_POR.pdf')

pdf_final = pyf.PdfWriter()
for pagina in pdf_rodar.pages:
    pagina.rotate(90)
    pdf_final.add_page(pagina)
    
with Path(r'Documentos PDF - Mentoria\Rodado.pdf').open(mode='wb') as arquivo_final:
        pdf_final.write(arquivo_final)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [6]:
arquivo = pyf.PdfReader(r'Documentos PDF - Mentoria\MGLU_ER_3T20_POR.pdf')

num_paginas = len(arquivo.pages)
print(num_paginas)

informacoes = arquivo.metadata
print(informacoes)

24
{'/Title': 'DESEMPENHO FINANCEIRO CONSOLIDADO', '/Author': 'an_rezende', '/Subject': 'Receita Bruta', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': "D:20201109183121-03'00'", '/ModDate': "D:20201109183121-03'00'", '/Producer': 'Microsoft® Office Word 2007'}


In [7]:
texto_procurado = '| Despesas com Vendas'

i = 1
for pagina in arquivo.pages:
    texto_pagina = pagina.extract_text()
    if texto_procurado in texto_pagina:
      print(f'Está na Página {i}')
      num_pagina = i
      texto_final = texto_pagina
    i += 1

Está na Página 10


In [8]:
texto_final = texto_final.replace('  ', ' ')
print(texto_final)

Divulgação de Resultados 
3T20 
10 
 
 
 
| Despesas Operacionais 
 
R$ milhões 3T20 
Ajustado  % RL 3T19 
Ajustado  % RL  Var(%) 9M20 
Ajustado  % RL 9M19 
Ajustado  % RL  Var(%) 
 Despesas com Vendas  (1.432,6) -17,2% (890,0) -18,3% 61,0% (3.487,2) -18,2% (2.309,1) -17,1% 51,0% 
 Despesas Gerais e Administrativas  (240,7) -2,9% (207,1) -4,3% 16,2% (617,3) -3,2% (498,2) -3,7% 23,9% 
 Subtotal   (1.673,3) -20,1%   (1.097,1) -22,6% 52,5%   (4.104,5) -21,5%   (2.807,4) -20,8% 46,2% 
 Perdas em Liquidação Duvidosa  (25,4) -0,3% (20,2) -0,4% 25,4% (84,5) -0,4% (45,8) -0,3% 84,3% 
 Outras Receitas Operacionais, Líquidas  15,2 0,2% 15,3 0,3% -0,6% 41,0 0,2% 44,0 0,3% -6,8% 
 Total de Despesas Operacionais    (1.683,5) -20,3%   (1.102,0) -22,7% 52,8%   (4.148,0) -21,7%   (2.809,2) -20,8% 47,7% 
 
| Despesas com Vendas 
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimen

In [10]:
posicao = texto_final.find('| Despesas com Vendas')
posicao_final = texto_final.find("|", posicao + 1)
texto_despesa = texto_final[posicao:posicao_final]
print(texto_despesa)

| Despesas com Vendas 
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com 
vendas m esmo investi ndo em maior nível de serviço, especialmente em atendimento e logística. 
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versus os 
9M19). 
 



#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [9]:
from pikepdf import Pdf, PdfImage

filename = "MGLU_ER_3T20_POR.pdf"
example = Pdf.open(filename)

n_arquivo = 0
for i, pagina in enumerate(example.pages):
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imgs/Pagina_{i:02}_{n_arquivo}')
        n_arquivo += 1

ModuleNotFoundError: No module named 'pikepdf'

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url